In [1]:
import numpy as np

A function for maximization of an arithmetical expression uzing Bellman's optimality principle . See [link](https://neerc.ifmo.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE_%D1%80%D0%B0%D1%81%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%B5_%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2_%D0%B2_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B8)

In [400]:
def get_result(current, e,result_expr,index_table, operators):
    if len(current) == 1:
        return result_expr
    l = current[0][1]
    r = current[len(current) - 1][1]
    result_expr.insert(result_expr.index(current[0]),'(')
    result_expr.insert(result_expr.index(current[len(current) - 1])+1,')')
    index = index_table[l,r]
    operator = operators[l,r]
    index_result = result_expr.index(e[index])
    result_expr.insert(index_result+1,operator)
    get_result(e[l:index+1],e, result_expr,index_table,operators)
    get_result(e[index+1:r+1], e, result_expr,index_table,operators)
    return result_expr  

In [405]:
# maximization of an expression expr with dynamic programming
def dynamic_maximize(expr):
    n = len(expr)
    d = np.zeros((n,n), dtype = int)
    index_table = np.zeros((n,n), dtype = int)
    operators = np.zeros((n,n), dtype = str)
    for i in range(n):
        d[i,i] = expr[i]
    for i in range(n-2,-1,-1):
        for j in range(i+1,n):
            for k in range(i,j):
                
                d_current = max(d[i,k]+d[k+1,j],d[i,k]*d[k+1,j])
                if d[i,j] < d_current:
                    d[i,j] = d_current
                    index_table[i,j] = k
                    if d[i,k]*d[k+1,j] >= d[i,k]+d[k+1,j]:
                        operators[i,j] = '*'
                    else:
                        operators[i,j] = '+' 
                
                #d[i,j] = max(d[i,j],max(d[i,k]+d[k+1,j],d[i,k]*d[k+1,j]))
    e = []
    n = len(expr)
    for i in range(n):
        e.append((expr[i],i))
    result_expr = e[:]
    result = get_result(e[:], e,result_expr,index_table,operators)
    res = [x[0] if type(x) is tuple else x for x in result]
    print(''.join(str(i) for i in res),'=',d[0,n-1])

In [406]:
dynamic_maximize([2,1,7,1,4,3])

((2+1)*(7*((1+4)*3))) = 315


In [418]:
# minimization of an expression expr with dynamic programming
def dynamic_minimize(expr):
    n = len(expr)
    d = np.zeros((n,n), dtype = int)
    index_table = np.zeros((n,n), dtype = int)
    operators = np.zeros((n,n), dtype = str)
    for i in range(n):
        d[i,i] = expr[i]
    for i in range(n-2,-1,-1):
        for j in range(i+1,n):
            for k in range(i,j):
                d_current = min(d[i,k]+d[k+1,j],d[i,k]*d[k+1,j])
                if d[i,j] > d_current or d[i,j] ==0:
                    d[i,j] = d_current
                    index_table[i,j] = k
                    if d[i,k]*d[k+1,j] < d[i,k]+d[k+1,j]:
                        operators[i,j] = '*'
                    else:
                        operators[i,j] = '+' 
    e = []
    n = len(expr)
    for i in range(n):
        e.append((expr[i],i))
    result_expr = e[:]
    result = get_result(e[:], e,result_expr,index_table,operators)
    res = [x[0] if type(x) is tuple else x for x in result]
    print(''.join(str(i) for i in res),'=',d[0,n-1])        

In [419]:
dynamic_minimize([2,1,7,1,4,3])

(2+(1*(7+(1*(4+3))))) = 16
